In [ ]:
import os
from pathlib import Path

from src.projects.fagradalsfjall.common.project_settings import (
    TS_PRIMARY_METRIC_DISPLAY_NAME,
    PATH_BLOG_POSTS,
)
from src.projects.fagradalsfjall.common.model_eval_plot_curves import generate_all_rmse_plots
from src.projects.fagradalsfjall.common.model_eval_plot_simulations import plot_simulations
from src.projects.fagradalsfjall.common.model_repo import ModelRepo
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult, ValidationType
from src.projects.fagradalsfjall.common.paths import get_blog_post_subfolder

from src.tools.datetime import format_timedelta

# 2. Visualization

In [ ]:
# -------------------------------------------------------------------------
#  Output path settings
# -------------------------------------------------------------------------
path_results_arma = get_blog_post_subfolder(4, "results_1_arma")
path_results_ar = get_blog_post_subfolder(4, "results_2_ar")
path_results_final = get_blog_post_subfolder(4, "results_3_final")

In [ ]:
# -------------------------------------------------------------------------
#  Load all data
# -------------------------------------------------------------------------

# --- load models -------------------------------------
naive_model_eval_results = ModelRepo.load_models(["naive-mean", "naive-constant", "naive-exp-decay"])
arma_model_eval_results = ModelRepo.load_models(["linear-arma"])
ar_model_eval_results = ModelRepo.load_models(["linear-ar"])

# --- post-processing ---------------------------------
model_arma = arma_model_eval_results["linear-arma"].ts_model
model_ar = ar_model_eval_results["linear-ar"].ts_model

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - ARMA - Cross-Validation
# -------------------------------------------------------------------------

# --- 2D plots ----------------------------------------
wd_values = model_arma.cv.results.all_param_values()["wd"]
for wd in sorted(wd_values):
    fig, ax = model_arma.cv.results.plot_2d(x_param="p", y_param="q", param_filter=dict(wd=wd)).create(w=12, h=8)
    fig.suptitle(f"Cross-validation results - 'linear-arma' - [wd={wd}]")

    wd_str = str(wd).replace(".", "_")
    fig.savefig(path_results_arma / f"cv_2d_pq_wd_{wd_str}.png", dpi=600)

# --- 1D plots - cross-section ----------------------------
p_opt = model_arma.cv.results.best_result.params["p"]
q_opt = model_arma.cv.results.best_result.params["q"]
wd_opt = model_arma.cv.results.best_result.params["wd"]

# cross-section sweep - p
fig, ax = (
    model_arma.cv.results.plot_1d(
        param_names="p",
        param_filter={"q": q_opt, "wd": wd_opt},
    )
    .set_fig_title(f"Cross-validation - 'linear-arma' - [q={q_opt}, wd={wd_opt}]")
    .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
    .create(w=12, h=8)
)
fig.savefig(path_results_arma / f"cv_1d_cross_section_p.png", dpi=600)

# cross-section sweep - q
fig, ax = (
    model_arma.cv.results.plot_1d(
        param_names="q",
        param_filter={"p": p_opt, "wd": wd_opt},
    )
    .set_fig_title(f"Cross-validation - 'linear-arma' - [p={p_opt}, wd={wd_opt}]")
    .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
    .create(w=12, h=8)
)
fig.savefig(path_results_arma / f"cv_1d_cross_section_q.png", dpi=600)

# cross-section sweep - wd
fig, ax = (
    model_arma.cv.results.plot_1d(
        param_names="wd",
        param_filter={"p": p_opt, "q": q_opt},
    )
    .set_fig_title(f"Cross-validation - 'linear-arma' - [p={p_opt}, q={q_opt}]")
    .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
    .create(w=12, h=8)
)
fig.savefig(path_results_arma / f"cv_1d_cross_section_wd.png", dpi=600)

# --- 1D plots - best -------------------------------------
for param_name in ["p", "q", "wd"]:

    fig, ax = (
        model_arma.cv.results.plot_1d(
            param_names=param_name,
        )
        .set_fig_title(f"Cross-validation - 'linear-arma' - best results for each value of '{param_name}'")
        .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
        .create(w=12, h=8)
    )

    fig.savefig(path_results_arma / f"cv_1d_best_{param_name}.png", dpi=600)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - ARMA - Simulations
# -------------------------------------------------------------------------

# RMSE curves
model_eval_results = naive_model_eval_results | arma_model_eval_results
generate_all_rmse_plots(model_eval_results, path_results_arma, highlight_models=["linear-arma"])

# simulations
plot_simulations(arma_model_eval_results, path_results_arma)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - AR - Cross-Validation
# -------------------------------------------------------------------------

# --- 2D plots ----------------------------------------
fig, ax = model_ar.cv.results.plot_2d(
    x_param="p",
    y_param="wd",
).create(w=12, h=8)
fig.savefig(path_results_ar / f"cv_2d_p_wd.png", dpi=600)

# --- 1D plots - cross-section ----------------------------
p_opt = model_ar.cv.results.best_result.params["p"]
wd_opt = model_ar.cv.results.best_result.params["wd"]

# cross-section sweep - p
fig, ax = (
    model_ar.cv.results.plot_1d(
        param_names="p",
        param_filter={"wd": wd_opt},
    )
    .set_fig_title(f"Cross-validation - 'linear-ar' - [wd={wd_opt}]")
    .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
    .create(w=12, h=8)
)

fig.savefig(path_results_ar / f"cv_1d_cross_section_p.png", dpi=600)

# cross-section sweep - wd
fig, ax = (
    model_ar.cv.results.plot_1d(
        param_names="wd",
        param_filter={"p": p_opt},
    )
    .set_fig_title(f"Cross-validation - 'linear-ar' - [p={p_opt}]")
    .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
    .create(w=12, h=8)
)
fig.savefig(path_results_ar / f"cv_1d_cross_section_wd.png", dpi=600)

# --- 1D plots - best -------------------------------------
for param_name in ["p", "wd"]:

    fig, ax = (
        model_ar.cv.results.plot_1d(
            param_names=param_name,
        )
        .set_fig_title(f"Cross-validation - 'linear-ar' - best results for each value of '{param_name}'")
        .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
        .create(w=12, h=8)
    )

    fig.savefig(path_results_ar / f"cv_1d_best_{param_name}.png", dpi=600)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - AR - Simulations
# -------------------------------------------------------------------------

# RMSE curves
model_eval_results = naive_model_eval_results | arma_model_eval_results | ar_model_eval_results
generate_all_rmse_plots(model_eval_results, path_results_ar, highlight_models=["ar"])

# simulations
plot_simulations(ar_model_eval_results, path_results_ar)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - FINAL - Simulations
# -------------------------------------------------------------------------

# RMSE curves
all_model_eval_results = naive_model_eval_results | arma_model_eval_results | ar_model_eval_results
generate_all_rmse_plots(all_model_eval_results, path_results_final, highlight_models=["linear-arma", "linear-ar"])

# simulations
plot_simulations(arma_model_eval_results, path_results_final)
plot_simulations(ar_model_eval_results, path_results_final)

In [ ]:
df_metrics = ModelEvalResult.all_metric_values_as_df(all_model_eval_results, ValidationType.TEST)

df_metrics["max_accurate_lead_time_samples"] = df_metrics["max_accurate_lead_time"].copy()
df_metrics["max_accurate_lead_time"] = df_metrics["max_accurate_lead_time"].apply(
    lambda x: format_timedelta(x * 15 * 60)
)

display(df_metrics)